Now that I've streamlined the MCMC process, I am going to submit multiple chains simultaneously. This notebook will make multiple, similar config files, for broad comparison. 

This may be rolled into pearce as a helper function, I haven't decided.

For rmin 0, 0.5, 1.0:

    For no ab, HSAB and CorrAB emu:

        Vpeak sham
        Mpeak sham
        HOD
        HSAB HOD 

In [12]:
import yaml
import copy
from os import path
import numpy as np

In [13]:
orig_cfg_fname = '/home/users/swmclau2/Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [14]:
orig_cfg

{'chain': {'fixed_params': {},
  'nburn': 0,
  'nsteps': 10000,
  'nwalkers': 500,
  'seed': 0},
 'data': {'cov': {'emu_cov_fname': '/home/users/swmclau2/Git/pearce/bin/covmat/xi_gg_nh_emu_cov.npy',
   'meas_cov_fname': '/home/users/swmclau2/Git/pearce/notebooks/meas_cov_testboxes_gg.npy'},
  'obs': {'mean': False,
   'obs': 'xi',
   'rbins': [0.07943282,
    0.11220185,
    0.15848932,
    0.22387211,
    0.31622777,
    0.44668359,
    0.63095734,
    0.89125094,
    1.25892541,
    1.77827941,
    2.51188643,
    3.54813389,
    5.01187234,
    7.07945784,
    10.0,
    14.12537545,
    19.95262315,
    28.18382931,
    39.81071706]},
  'sim': {'gal_type': 'SHAM',
   'halo_property': 'halo_vpeak',
   'min_ptcl': 100,
   'nd': '5e-4',
   'scale_factor': 1.001,
   'scatter': 0.2,
   'sim_hps': {'system': 'sherlock'},
   'simname': 'ds_14_b_sub'},
  'true_cov_fname': ['/home/users/swmclau2/Git/pearce/bin/shams/xigg_cov_mcmc.npy'],
  'true_data_fname': ['/home/users/swmclau2/Git/pearce/

In [15]:
orig_sbatch_fname = '/home/users/swmclau2/Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.sbatch'
with open(orig_sbatch_fname, 'r') as f:
    lines = []
    for line in f:
        lines.append(line)
        
orig_sbatch = ''.join(lines)

In [16]:
#this will enable easier string formatting
sbatch_template = """#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --time=8:00:00
#SBATCH -p kipac
#SBATCH -o /home/users/swmclau2/Git/pearce/bin/mcmc/config/{jobname}.out
#SBATCH --ntasks=16
###SBATCH --exclusive

module load python/2.7.13
module load py-scipystack
module load hdf5/1.10.0p1
python /home/users/swmclau2/Git/pearce/pearce/inference/initialize_mcmc.py {jobname}.yaml
python /home/users/swmclau2/Git/pearce/pearce/inference/run_mcmc.py {jobname}.yaml
"""

In [17]:
#emu fnames
# only sending out the HOD one cuz the others aren't yet finished
emu_fnames = ['/scratch/users/swmclau2/xi_gg_zheng07/PearceXiggCosmo.hdf5',\
              '/scratch/users/swmclau2/xi_gg_corrab_zheng07/PearceXiggCosmoCorrAB.hdf5',
                '/scratch/users/swmclau2/xi_gg_hsab_zheng07/PearceXiggHSABCosmo.hdf5']

emu_cov_fnames = ['/home/users/swmclau2/Git/pearce/bin/covmat/xi_gg_nh_emu_cov_v4.npy',
                  '/home/users/swmclau2/Git/pearce/bin/covmat/xi_gg_nh_emu_corrab_cov_v4.npy',
                    '/home/users/swmclau2/Git/pearce/bin/covmat/xi_gg_nh_emu_hsab_cov_v4.npy']
        

emu_names = ['HOD', 'CorrAB', 'HSAB']

In [18]:
#meas_cov = np.load('/home/users/swmclau2/Git/pearce/notebooks/meas_cov_testboxes_gg.npy')
meas_cov = np.load('/home/users/swmclau2/Git/pearce/bin/covmat/xi_gg_darksky_cov.npy')

In [19]:
# prep full covs
full_covs = []
for emu_name, emu_fname, cov_fname in zip(emu_names, emu_fnames, emu_cov_fnames):
    cov = np.load(cov_fname)
    
    full_cov = cov+meas_cov
    fname = '/home/users/swmclau2/Git/pearce/notebooks/%s_full_cov.npy'%emu_name
    np.savetxt(fname, full_cov)
    full_covs.append(fname)

## Vpeak SHAM
----


In [20]:
tmp_cfg = copy.deepcopy(orig_cfg)
directory = "/home/users/swmclau2/Git/pearce/bin/mcmc/config/"
output_dir = "/home/users/swmclau2/scratch/PearceMCMC/"
jobname_template = "VpeakSHAM_xi_gg_rmin_{rmin}_{emu_name}_v5"

for rmin in [None, 0.5, 1.0]:
    for emu_fname, emu_name, emu_cov in zip(emu_fnames, emu_names, full_covs):
        
        if rmin is not None:
            tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}
            
        tmp_cfg['emu']['training_file'] = emu_fname
        
        tmp_cfg['data']['true_data_fname']= ['/home/users/swmclau2/Git/pearce/bin/shams/ds14b_sub_xi_gg.npy']
        tmp_cfg['data']['true_cov_fname'] = [emu_cov]
        tmp_cfg['chain']['nsteps'] = 20000
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        with open(path.join(directory, jobname + '.sbatch'), 'w') as f:
            f.write(sbatch_template.format(jobname=jobname))

## Mpeak SHAM
----


## HOD
----


In [21]:
orig_cfg_fname = '/home/users/swmclau2/Git/pearce/bin/mcmc/nh_gg_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [22]:
tmp_cfg = copy.deepcopy(orig_cfg)
directory = "/home/users/swmclau2/Git/pearce/bin/mcmc/config/"
output_dir = "/home/users/swmclau2/scratch/PearceMCMC/"
jobname_template = "HOD_xi_gg_rmin_{rmin}_{emu_name}_v5"

for rmin in [None, 0.5, 1.0]:
    for emu_fname, emu_name, emu_cov in zip(emu_fnames, emu_names, emu_cov_fnames):
        
        tmp_cfg['chain']['nwalkers'] = 500
        if rmin is not None:
            tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}
            
        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['data']['cov']['emu_cov_fname'] = emu_cov
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')
        
        tmp_cfg['sim']= {'gal_type': 'HOD',
       'hod_name': 'zheng07',
       'hod_params': {'alpha': 1.083,
        'logM0': 13.2,
        'logM1': 14.2,
        'sigma_logM': 0.2},
       'nd': '5e-4',
       'scale_factor': 1.0,
       'sim_hps': {'boxno': 1,
        'downsample_factor': '1e-2',
        'particles': False,
        'realization': 0,
        'system': 'sherlock'},
       'simname': 'testbox'}
        
        tmp_cfg['chain']['nsteps'] = 20000

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        with open(path.join(directory, jobname + '.sbatch'), 'w') as f:
            f.write(sbatch_template.format(jobname=jobname))

## HSAB HOD
----
